In [46]:
# import dependencies

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from pandas import DataFrame

In [47]:
html = "https://www.zip-codes.com/county/ca-san-diego.asp"
contents = requests.get(html)

soup = BeautifulSoup(contents.content, 'lxml')
# Need to parse through DOM to know which table to scrape
table = soup.find_all('table')[2]
lists = pd.read_html(str(table))

In [48]:
website_data_df = lists[0]
print(f'Initial size of dataframe: {website_data_df.shape}')
# Set column headers
website_data_df.columns = website_data_df.iloc[0]
website_data_df = website_data_df.drop(0)
# Reset index 
website_data_df = website_data_df.reset_index(drop=True)
website_data_df.head()

Initial size of dataframe: (182, 6)


,ZIP Code,Classification,City,Population,Timezone,Area Code(s)
0,ZIP Code 91901,General,Alpine,17403,Pacific,619
1,ZIP Code 91902,General,Bonita,17653,Pacific,619
2,ZIP Code 91903,P.O. Box,Alpine,0,Pacific,619
3,ZIP Code 91905,General,Boulevard,1700,Pacific,619/760
4,ZIP Code 91906,General,Campo,3627,Pacific,619


In [49]:
# Drop Zip Codes with Population 0
no_population = website_data_df[website_data_df["Population"] == '0'].index
website_data_df.drop(no_population, inplace=True)
print(f'Updated size of dataframe: {website_data_df.shape[0]} rows and {website_data_df.shape[1]} columns')
# Drop "Zip Code" phrase in Zip Code column
website_data_df["ZIP Code"] = website_data_df["ZIP Code"].str[-5:]
website_data_df.head()

Updated size of dataframe: 106 rows and 6 columns


,ZIP Code,Classification,City,Population,Timezone,Area Code(s)
0,91901,General,Alpine,17403,Pacific,619
1,91902,General,Bonita,17653,Pacific,619
3,91905,General,Boulevard,1700,Pacific,619/760
4,91906,General,Campo,3627,Pacific,619
7,91910,General,Chula Vista,75802,Pacific,619


In [51]:
# Dropping rows messed with the index. Time to unmess it up!
df = website_data_df.reset_index(drop=True)
df.rename(columns={"ZIP Code": "Code"}, inplace=True)

df.head(5)

,Code,Classification,City,Population,Timezone,Area Code(s)
0,91901,General,Alpine,17403,Pacific,619
1,91902,General,Bonita,17653,Pacific,619
2,91905,General,Boulevard,1700,Pacific,619/760
3,91906,General,Campo,3627,Pacific,619
4,91910,General,Chula Vista,75802,Pacific,619


In [52]:
import geopy
from geopy.geocoders import  Nominatim

geolocator = Nominatim(user_agent="sd_explorer")


lats = []
longs = []

for zip in df["Code"]:
    address = zip
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    lats.append(latitude)
    longs.append(longitude)

In [53]:
df["Latitude"] = lats
df["Longitude"] = longs

In [17]:
# Install and import folium as map rendering library
!conda install -c conda-forge folium=0.5.0 --yes
import folium

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Import k-means from clustering stage from SciKit-Learn

from sklearn.cluster import KMeans

Solving environment: done

# All requested packages already installed.



In [75]:
# df.drop(columns=['Classification', 'Timezone'], inplace=True)
df.head()

,Code,City,Population,Area Code(s),Latitude,Longitude
0,91901,Alpine,17403,619,48.414003,17.491107
1,91902,Bonita,17653,619,32.666379,-117.022725
2,91905,Boulevard,1700,619/760,32.662786,-116.312998
3,91906,Campo,3627,619,32.662692,-116.481502
4,91910,Chula Vista,75802,619,48.534285,2.181133


In [55]:
address = 'San Diego, CA, USA'

geolocator = Nominatim(user_agent="sd_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of San Diego are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of San Diego are 32.7174209, -117.1627714.


In [79]:
zip(df['Latitude'])

TypeError: 'str' object is not callable

In [64]:
map_sd = folium.Map(location=[latitude, longitude], zoom_start = 11)

# # add markers to map

for lat, long, code, city in zip(sd_lat, sd_long, sd_code, sd_city):
    label = '{}, {}'.format(city, code)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sd)

map_sd

TypeError: 'str' object is not callable

In [65]:
sd_lat = df['Latitude'].tolist()
sd_long = df['Longitude'].tolist()
sd_code = df['Code'].tolist()
sd_city = df['City'].tolist()

In [70]:
zipped = zip(sd_lat)

TypeError: 'str' object is not callable